In [19]:
import numpy as np
np.set_printoptions(suppress=True)

In [20]:
class Zonotope:
    def __init__(self, centre, generators):
        self.centre = np.array(centre).reshape(-1)         # shape (n,)
        self.generators = np.atleast_2d(generators)        # shape (n, m)
        if self.generators.shape[0] != self.centre.shape[0]:
            raise ValueError("Generators must have same dimension as centre")
        self.d = self.centre.shape[0]   # dimension
        self.m = self.generators.shape[1]  # number of generators

    def minkowski_sum(self, other):
        new_centre = self.centre + other.centre
        new_generators = np.hstack((self.generators, other.generators))
        return Zonotope(new_centre, new_generators)
    
    def subtract(self, other):
        new_centre = self.centre - other.centre
        new_generators = np.hstack((self.generators, -other.generators))
        return Zonotope(new_centre, new_generators)
    
    @classmethod
    def from_intervals(cls, lower, upper):
        lower = np.array(lower).reshape(-1)
        upper = np.array(upper).reshape(-1)

        if lower.shape != upper.shape:
            raise ValueError("Lower and upper must have same shape")

        centre = (lower + upper) / 2
        radii = (upper - lower) / 2

        generators = np.diag(radii)
        return cls(centre, generators)
    
    def output_interval(self):
        radius = np.sum(np.abs(self.generators), axis=1)  # sum across generators
        lower = self.centre - radius
        upper = self.centre + radius
        return lower, upper

    def affine_map(self, A, b=None):
            # Ensure centre is a column vector for matrix multiplication
            new_centre = A @ self.centre
            if b is not None:
                new_centre = new_centre + np.array(b).reshape(-1)
            new_generators = A @ self.generators
            return Zonotope(new_centre, new_generators)
    
    def __repr__(self):
        return f"Zonotope(dim={self.d}, generators={self.m})\nCentre:\n{self.centre}\nGenerators:\n{self.generators}"

### 1. Initial Zonotope

In [21]:
y1 = [1.0, 2.0]
y2 = [2.0, 4.0]
lower = [y1[0], y2[0]]
upper = [y1[1], y2[1]]
X = np.array([[1, 1], 
              [1, 2]])


Z_y = Zonotope.from_intervals(lower, upper)
print(Z_y)


Zonotope(dim=2, generators=2)
Centre:
[1.5 3. ]
Generators:
[[0.5 0. ]
 [0.  1. ]]


### 2. First Affine Transformation

In [22]:
M = np.linalg.inv(X.T @ X) @ X.T
print(M)

Z_beta = Z_y.affine_map(M)
print(Z_beta)

[[ 2. -1.]
 [-1.  1.]]
Zonotope(dim=2, generators=2)
Centre:
[0.  1.5]
Generators:
[[ 1.  -1. ]
 [-0.5  1. ]]


### 3. Second Affine Transformation

In [23]:
X_g = np.array([[1, 0],
                [1, 1],
                [1, 3]])

Z_yhat = Z_beta.affine_map(X_g)
print(Z_yhat)

Zonotope(dim=3, generators=2)
Centre:
[0.  1.5 4.5]
Generators:
[[ 1.  -1. ]
 [ 0.5  0. ]
 [-0.5  2. ]]


### 4. Output Intervals


In [ ]:
Z_yhat_lower, Z_yhat_upper = Z_yhat.output_interval()

print(f'y_x1 : [{Z_yhat_lower[0]}, {Z_yhat_upper[0]}]')
print(f'y_x2 : [{Z_yhat_lower[1]}, {Z_yhat_upper[1]}]')
print(f'y_x3 : [{Z_yhat_lower[2]}, {Z_yhat_upper[2]}]')

y_x1 -1.9999999999999991 2.0000000000000027
y_x2 1.0000000000000002 2.000000000000001
y_x3 1.9999999999999973 6.999999999999999
